# 9-Distill

模型蒸馏 (Knowledge Distillation, KD) 是一种机器学习模型压缩方法，它用于将大型模型（教师模型）的知识迁移到较小的模型（学生模型）中.

KD 背后的核心思想是将教师模型的综合知识转化为更精简、更有效的表示. 学生模型是一个较小的模型，目标是学习教师模型的行为，而不是直接从原始数据中学习.

大模型的 KD 有白盒蒸馏与黑盒蒸馏两个派别，对于本次实验代码中两个模型均为 MiniMind 开源模型，支持对教师模型内部结构的访问，因此在训练过程中，我们能够获取教师模型的 softmax 概率分布并用作软标签（soft labels），让小模型学习软标签，并使用 KL-Loss 来优化模型的参数，而不是直接学习输出 Token 的硬标签. 对于下一章蒸馏推理模型中，由于我们面向推理数据集进行蒸馏，并不存在输出 Token 的概率分布让我们学习，这种面向输出数据学习的蒸馏方式被称为黑盒蒸馏.

此笔记本的完整实现见主仓库 `/minimind/train_distillation.py`

In [1]:
import os
import argparse
import time
import math
import warnings

import pandas as pd
import torch
import torch.nn.functional as F
import torch.distributed as dist
from contextlib import nullcontext

from torch import optim, nn
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data import DataLoader, DistributedSampler
from transformers import AutoTokenizer, AutoModelForCausalLM
from model.model import MiniMindLM
from model.LMConfig import LMConfig
from model.dataset import SFTDataset

warnings.filterwarnings('ignore')

## 可选参数设置

首先，查看训练的可选参数，这些参数在实际使用时通过解析命令行进行导入，我们用 class 进行包装.

In [2]:
class args:
    epochs: int = 5 # 训练轮数，延续 pretrain 基础上微调
    batch_size: int = 2 # pretrain 数据集仅两个样本，设置 batch 为 2
    learning_rate: float = 5e-4 # 学习率
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype: str = 'bfloat16' # 16 bit 浮点数：8 bit 指数 + 7 bit 尾数
    # use_wandb: bool = False # 是否使用 wandb 我们不使用
    wandb_project: str = 'MiniMind-Notebook'
    num_workers: int = 1 # 工作进程数
    # ddp：bool = False # 单机多卡
    accumulation_steps: int = 1 # 梯度累积步数
    grad_clip: float = 1.0 # 梯度剪裁
    warmup_iters: int = 0 # 学习率热启动
    log_interval: int = 1 # 每一步打印日志 仅用于观察
    local_rank: int = 1 # device 设备号
    dim: int = 512 # 词嵌入维度 模型超参数
    n_layers: int = 2 # MiniMind Block 数量 模型超参数
    max_seq_len: int = 512 # 序列长度阈值
    use_moe: bool = False # 是否启用混合专家
    data_path: str = './toydata/sft_data.jsonl' # 数据集路径
    save_dir: str = "./output"  # 模型保存目录
    save_weight: str = "minimind_distill"  # checkpoint 文件前缀
    save_interval: int = 1  # 每多少步保存一次模型，0表示不保存 我们这里只展示训练过程（可选择的保存模型，建议先保存）

In [3]:
print(f'查看工作设备 {args.device}')

查看工作设备 cuda


接下来，我们对分词器、MiniMind 教师/学生模型以及数据迭代器执行初始化.

In [ ]:
def init_student_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('./model/minimind_tokenizer')
    model = MiniMindLM(lm_config)
    moe_path = '_moe' if lm_config.use_moe else ''
    # 学生模型热启动
    ckp = f'./output/minimind_sft_{lm_config.dim}{moe_path}.pth'
    # ckp = f'./output/minimind_rlaif_{lm_config.dim}{moe_path}.pth' 或者可以加载rlaif的权重继续训练
    state_dict = torch.load(ckp, map_location=args.device)
    model.load_state_dict(state_dict, strict=False)
    print(f'学生模型(LLM)总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    model = model.to(args.device)
    # 学生模型：返回模型+分词器
    return model, tokenizer


def init_teacher_model(lm_config):
    model = MiniMindLM(lm_config)
    moe_path = '_moe' if lm_config.use_moe else ''
    # 教师模型热启动
    ckp = f'./output/minimind_sft_{lm_config.dim}{moe_path}.pth'
    # ckp = f'./output/minimind_rlaif_{lm_config.dim}{moe_path}.pth' 或者可以加载rlaif的权重继续训练
    state_dict = torch.load(ckp, map_location=args.device)
    model.load_state_dict(state_dict, strict=False)
    print(f'教师模型(LLM)总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    model = model.to(args.device)
    # 教师模型：仅返回模型
    return model

In [ ]:
# 初始化模型配置，一般学生模型比较小，从大的教师模型那里学知识
# 需要提前训练好教师模型，保存 checkpoint，学生模型热启动教师模型的权重（不完全加载，允许缺失），在此基础上继续训练学生模型
# 即在 pretrain 和 sft 或 rlaif 阶段训练得到 768 维的教师模型
lm_config_student = LMConfig(dim=512, n_layers=1, max_seq_len=512)
lm_config_teacher = LMConfig(dim=768, n_layers=2, max_seq_len=512)

model, tokenizer = init_student_model(lm_config_student)
teacher_model = init_teacher_model(lm_config_teacher)

# 初始化数据集和 DataLoader
train_ds = SFTDataset(args.data_path, tokenizer, max_length=lm_config_student.max_seq_len)
train_loader = DataLoader(
    train_ds,
    batch_size=args.batch_size,
    pin_memory=True,
    drop_last=False,
    shuffle=False,
    num_workers=args.num_workers,
)

print(f'模型位于设备：{model.device}, 词表长度：{tokenizer.vocab_size}, DataLoader：{train_loader}')

学生模型(LLM)总参数量：6.096 百万
教师模型(LLM)总参数量：17.305 百万
模型位于设备：cuda:0, 词表长度：6400, DataLoader：<torch.utils.data.dataloader.DataLoader object at 0x000001A3210C9DD0>


## 启动训练

接下来，我们定义 MiniMind LoRA 微调所使用的优化器，损失函数和学习率调度，并进行一轮简单的训练.

In [6]:
# 学习率调度方面 采用余弦退火学习率
def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

# 优化器方面 选择 AdamW 优化器 并在混精度场景下创建 scaler 进行梯度缩放避免数值下溢
scaler = torch.amp.GradScaler('cuda', enabled=(args.dtype in ['float16', 'bfloat16']))  # 专门解决混合精度训练中的数值下溢问题
# 优化学生模型参数
optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)

device_type = "cuda" if "cuda" in args.device else "cpu"
print(f'设备类型：{device_type}')
# 根据指定的数据类型设置混精度训练的 dtype，以下步骤为不可缺少的混精度训练准备工作
if args.dtype == 'bfloat16':
    amp_dtype = torch.bfloat16
elif args.dtype == 'float16':
    amp_dtype = torch.float16
else:
    amp_dtype = torch.float32  # 默认为 FP32
print(f'使用混精度训练，数据类型：{amp_dtype}')
# 在 cuda 上启动混精度训练，否则空白上下文
autocast_ctx = nullcontext() if device_type == "cpu" else torch.amp.autocast(device_type='cuda', dtype=amp_dtype) 

设备类型：cuda
使用混精度训练，数据类型：torch.bfloat16


损失函数方面，使用 KL Loss 方法. 

KL Loss 中，损失是 KL 散度，衡量学生模型和教师模型在面对相同输入时，在输出层产生的分类 logits 分布之间的距离. 直观理解上，就是让学生模型的输出尽量向教师模型的输出概率靠近.

$$D_{KL}(P||Q)=\sum_i P(i)\log\frac{P(i)}{Q(i)}$$

其中，$P(i)$ 代表教师模型的概率分布，$Q(i)$ 代表学生模型的预测分布.

In [7]:
# 定义蒸馏损失函数， batchmean 表示对各批次损失求平均值
# temperature 参数用于控制教师模型输出概率分布的平滑程度，推荐值为1.0到2.0之间，较高的温度会使分布更平滑，较低的温度会使分布更尖锐
def distillation_loss_fn(student_logits, teacher_logits, temperature=1.0, reduction='batchmean'):
    # 对最后一维进行 softmax 计算概率分布，并使用 detach() 将其从计算图中分离出来，避免梯度传播到教师模型
    with torch.no_grad():
        teacher_probs = F.softmax(teacher_logits / temperature, dim=-1).detach() 

    # 对学生模型的 logits 进行 log_softmax 计算，得到 log 概率分布
    student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)

    kl = F.kl_div(
        student_log_probs,
        teacher_probs,
        reduction=reduction  # 对各批次损失求平均值
    )
    return (temperature ** 2) * kl # 尺度不变

接下来，我们来看训练函数.

In [8]:
def train_epoch(epoch, loader, iters, teacher_model, lm_config_student, start_step=0, wandb=None, alpha=0.0, temperature=1.0):
    start_time = time.time()
    
    if teacher_model is not None:
        teacher_model.eval()
        teacher_model.requires_grad_(False)

    for step, (X, Y, loss_mask) in enumerate(loader, start=start_step + 1):
        # 将输入数据移动到指定设备
        X = X.to(args.device)
        Y = Y.to(args.device)
        loss_mask = loss_mask.to(args.device)
        lr = get_lr(epoch * iters + step, args.epochs * iters, args.learning_rate)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # 前向传播（学生模型）
        with autocast_ctx:
            res = model(X)  # 前向推理
            student_logits = res.logits[..., :-1, :].contiguous()  # [batch, seq_len-1, vocab]

        # 教师模型前向传播（只在eval & no_grad）
        if teacher_model is not None:
            with torch.no_grad():
                teacher_logits = teacher_model(X).logits[..., :-1, :].contiguous()  # [batch, seq_len-1, vocab_teacher]
                vocab_size_student = student_logits.size(-1)   # 确保教师模型的输出维度与学生模型一致（如果教师模型词表更大，则截断）
                teacher_logits = teacher_logits[..., :vocab_size_student]  # 截断教师模型输出以匹配学生模型的词表大小

        # 计算损失
        # 1. 真实交叉熵损失
        shift_labels = Y[..., 1:].contiguous()  # [batch, seq_len-1]
        shift_loss_mask = loss_mask[..., 1:].contiguous()  # [batch, seq_len-1]
        loss_mask_flat = shift_loss_mask.view(-1)  # [batch * (seq_len-1)]
        ce_loss = F.cross_entropy(
            student_logits.view(-1, student_logits.size(-1)),
            shift_labels.view(-1),
            ignore_index=-100,  # 忽略标签值为 -100 的位置，这些位置不会对损失计算产生影响
            reduction='none'  # 计算每个位置的损失
        )
        ce_loss_raw = torch.sum(ce_loss * loss_mask_flat) / (loss_mask_flat.sum() + 1e-8)
        if lm_config_student.use_moe: ce_loss = ce_loss_raw + res.aux_loss
        else: ce_loss = ce_loss_raw

        # 2. 蒸馏损失
        if teacher_model is not None:
            distill_loss = distillation_loss_fn(
                student_logits.view(-1, student_logits.size(-1))[loss_mask_flat == 1],  # [num_tokens, vocab]
                teacher_logits.view(-1, teacher_logits.size(-1))[loss_mask_flat == 1],  # [num_tokens, vocab_teacher]
                temperature=temperature
            )
        else:
            distill_loss = torch.tensor(0.0, device=args.device)

        # 3. 总损失 = alpha * 真实交叉熵损失 + (1-alpha) * 蒸馏损失
        loss = (alpha * ce_loss + (1 - alpha) * distill_loss) / args.accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer) 
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        if step % args.log_interval == 0 or step == iters - 1:
            spend_time = time.time() - start_time
            current_loss = loss.item() * args.accumulation_steps
            current_ce_loss = ce_loss_raw.item()
            current_aux_loss = res.aux_loss.item() if lm_config_student.use_moe else 0.0
            current_lr = optimizer.param_groups[-1]['lr']
            eta_min = spend_time / (step + 1) * iters // 60 - spend_time // 60
            
            print(
                f'Epoch:[{epoch + 1}/{args.epochs}]({step}/{iters}), '
                f'loss: {current_loss:.4f}, '
                f'ce: {current_ce_loss:.4f}, '
                f'aux_loss: {current_aux_loss:.4f}, '
                f'distill: {distill_loss.item():.4f}, '
                f'learning_rate: {current_lr:.8f}, '
                f'epoch_time: {eta_min:.3f}min'
            )
            
            if wandb:
                wandb.log({
                    "loss": current_loss,
                    "ce_loss": current_ce_loss,
                    "aux_loss": current_aux_loss,
                    "distill_loss": distill_loss.item() if teacher_model is not None else 0.0,
                    "learning_rate": current_lr,
                    "epoch_time": eta_min
                })

        # 到达指定保存步数时，保存模型（仅主进程）
        if args.save_interval > 0 and (step % args.save_interval == 0 or step == iters - 1):
            if not dist.is_initialized() or dist.get_rank() == 0:
                os.makedirs(args.save_dir, exist_ok=True)  # 确保保存目录存在
                model.eval()
                moe_suffix = '_moe' if lm_config_student.use_moe else ''
                ckp = f'{args.save_dir}/{args.save_weight}_{lm_config_student.dim}{moe_suffix}.pth'
                raw_model = model.module if isinstance(model, DistributedDataParallel) else model
                raw_model = getattr(raw_model, '_orig_mod', raw_model)
                state_dict = raw_model.state_dict()
                torch.save({k: v.half().cpu() for k, v in state_dict.items()}, ckp)
                print(f'模型已保存至：{ckp}')
                model.train()
                del state_dict

        del X, Y, loss_mask, res, student_logits, ce_loss, distill_loss, loss

接下来，我们启动一个 Epoch 的训练进行观察.

In [9]:
iter_per_epoch = len(train_loader)
for epoch in range(args.epochs):
    train_epoch(epoch, train_loader, iter_per_epoch, teacher_model, lm_config_student, alpha=0.5, temperature=1.5)
print('蒸馏训练完成！')

Epoch:[1/5](1/1), loss: 3.0854, ce: 5.7600, aux_loss: 0.0000, distill: 0.4109, learning_rate: 0.00050225, epoch_time: 0.000min
模型已保存至：./output/minimind_distill_512.pth
Epoch:[2/5](1/1), loss: 1.9844, ce: 3.5800, aux_loss: 0.0000, distill: 0.3887, learning_rate: 0.00037725, epoch_time: 0.000min
模型已保存至：./output/minimind_distill_512.pth
Epoch:[3/5](1/1), loss: 1.2581, ce: 2.0900, aux_loss: 0.0000, distill: 0.4263, learning_rate: 0.00022275, epoch_time: 0.000min
模型已保存至：./output/minimind_distill_512.pth
Epoch:[4/5](1/1), loss: 0.9374, ce: 1.4200, aux_loss: 0.0000, distill: 0.4547, learning_rate: 0.00009775, epoch_time: 0.000min
模型已保存至：./output/minimind_distill_512.pth
Epoch:[5/5](1/1), loss: 0.8153, ce: 1.1600, aux_loss: 0.0000, distill: 0.4705, learning_rate: 0.00005000, epoch_time: 0.000min
模型已保存至：./output/minimind_distill_512.pth
蒸馏训练完成！


In [10]:
del model, teacher_model

## 参考资料

- [大模型知识蒸馏概述](https://zhuanlan.zhihu.com/p/659943824)
- [使用知识蒸馏将大模型能力克隆到小模型](https://zhuanlan.zhihu.com/p/691672620)
- [理解知识蒸馏中的散度损失函数](https://deepseek.csdn.net/67ab1c3f79aaf67875cb9664.html)